# Two way fixed effects con tutte le route, ma senza covariate

In [2]:
!pip install pandas==1.3.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 29.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.2.0 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.3.5 which is incompatible.
linearmodels 6.0 requires pandas>=1.4.0, but you have pandas 1.3.5 which is incompatible.
plotnine 0.12.4 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
statsmodels 0.14.2 requires pandas!=2.1.0,>=1.4, but you have pandas 1.3.5 which is incompatible.
xarray 2023.7.0 requires pandas>=1.4, but you have pandas 1.3.5 which is incompatible.


In [1]:
!pip install linearmodels
!pip install regtabletotext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import datetime as dt
import itertools
import linearmodels as lm

from regtabletotext import prettify_result, prettify_result

In [ ]:
df= pd.read_csv("regression_att_ALL_14_17.csv")

In [ ]:
df['Route'] = df['Route'].replace({'Central Mediterranean Route': 'central',
                                                                         'Eastern Mediterranean Route': 'eastern',
                                                                         'Western Mediterranean Route': 'western_m',
                                                                         'Western African Route': 'western_af'})


In [ ]:
df = df.rename(columns={'Total Number of Dead and Missing': 'Total_Deaths'})
df['date'] = pd.to_datetime(df['date'])
df.head()

,Route,date,flow,Total_Deaths,after,deaths_over_cross,ATT
0,central,2014-02-01,3335,9.0,0.0,0.002691,1.069824
1,central,2014-03-01,5550,1.0,0.0,0.000180,-4.842315
2,central,2014-04-01,15679,41.0,0.0,0.002608,-36.632737
3,central,2014-05-01,14597,299.0,0.0,0.020073,13.452974
4,central,2014-06-01,22778,314.0,0.0,0.013598,12.412410


In [ ]:
model_ols = (lm.PanelOLS.from_formula(
    formula="Total_Deaths ~ flow + ATT + 1",
    data=df.set_index(["Route", "date"]),
  ).fit()
)
prettify_result(model_ols)

Panel OLS Model:
Total_Deaths ~ flow + ATT + 1

Covariance Type: Unadjusted

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept    62.662      17.201        3.643    0.000
flow          0.001       0.000        1.938    0.055
ATT           1.488       0.246        6.038    0.000

Summary statistics:
- Number of observations: 144
- R-squared (incl. FE): 0.240, Within R-squared: 0.049



As expected, the regression output shows significant coefficients. However, the simple model actually may have a lot of omitted variables, so our coefficients are most likely biased. As there is a lot of unexplained variation in our simple model (indicated by the rather low adjusted R-squared).

One way to tackle the issue of omitted variable bias is to get rid of as much unexplained variation as possible by including fixed effects - i.e., model parameters that are fixed for specific Routes.

In [ ]:
model_ols = (lm.PanelOLS.from_formula(
    formula="Total_Deaths ~ flow + ATT + EntityEffects",
    data=df.set_index(["Route", "date"]),
  ).fit()
)
prettify_result(model_ols)

Panel OLS Model:
Total_Deaths ~ flow + ATT + EntityEffects

Covariance Type: Unadjusted

Coefficients:
      Estimate  Std. Error  t-Statistic  p-Value
flow     0.001       0.000        1.964    0.052
ATT      0.762       0.256        2.975    0.004

Included Fixed Effects:
        Total
Entity      4

Summary statistics:
- Number of observations: 144
- R-squared (incl. FE): 0.407, Within R-squared: 0.104



The regression output shows a lot of unexplained variation at the route level that is taken care of by including the route fixed effect as the R-squared rises by 0.2!.

By including year fixed effects, we can take out the effect of unobservables that vary over time. The two-way fixed effects regression.

In [ ]:
model_ols = (lm.PanelOLS.from_formula(
    formula="Total_Deaths ~ flow + ATT + EntityEffects + TimeEffects",
    data=df.set_index(["Route", "date"]),
  ).fit()
)
prettify_result(model_ols)

Panel OLS Model:
Total_Deaths ~ flow + ATT + EntityEffects + TimeEffects

Covariance Type: Unadjusted

Coefficients:
      Estimate  Std. Error  t-Statistic  p-Value
flow     0.001       0.001        2.072    0.041
ATT      0.709       0.288        2.456    0.016

Included Fixed Effects:
        Total
Entity      4
Time       37

Summary statistics:
- Number of observations: 144
- R-squared (incl. FE): 0.557, Within R-squared: 0.102



##Clustering Standard Errors

Apart from biased estimators, we usually have to deal with potentially complex dependencies of our residuals with each other. Such dependencies in the residuals invalidate the i.i.d. assumption of OLS and lead to biased standard errors. With biased OLS standard errors, we cannot reliably interpret the statistical significance of our estimated coefficients.

In [ ]:
# the code chunk below applies one-way clustering by Route

model_ols = lm.PanelOLS.from_formula(
    formula="Total_Deaths ~ flow + ATT + EntityEffects + TimeEffects",
    data=df.set_index(["Route", "date"]),
  ).fit(cov_type="clustered", cluster_entity=True, cluster_time=False)

prettify_result(model_ols)

Panel OLS Model:
Total_Deaths ~ flow + ATT + EntityEffects + TimeEffects

Covariance Type: Clustered

Coefficients:
      Estimate  Std. Error  t-Statistic  p-Value
flow     0.001       0.000        2.547    0.012
ATT      0.709       0.056       12.642    0.000

Included Fixed Effects:
        Total
Entity      4
Time       37

Summary statistics:
- Number of observations: 144
- R-squared (incl. FE): 0.557, Within R-squared: 0.102



In [ ]:
# two-way fe

# the code chunk below applies one-way clustering by Route

model_ols = lm.PanelOLS.from_formula(
    formula="Total_Deaths ~ flow + ATT + EntityEffects + TimeEffects",
    data=df.set_index(["Route", "date"]),
  ).fit(cov_type="clustered", cluster_entity=True, cluster_time=True)

prettify_result(model_ols)

Panel OLS Model:
Total_Deaths ~ flow + ATT + EntityEffects + TimeEffects

Covariance Type: Clustered

Coefficients:
      Estimate  Std. Error  t-Statistic  p-Value
flow     0.001       0.001        2.388    0.019
ATT      0.709       0.332        2.136    0.035

Included Fixed Effects:
        Total
Entity      4
Time       37

Summary statistics:
- Number of observations: 144
- R-squared (incl. FE): 0.557, Within R-squared: 0.102



# TWFE with conflicts

There are some simmgs coming from the Weastern AF

In [ ]:
df= pd.read_csv("regression_att_ALL_14_17.csv")

In [ ]:
df['Route'] = df['Route'].replace({'Central Mediterranean Route': 'central',
                                                                         'Eastern Mediterranean Route': 'eastern',
                                                                         'Western Mediterranean Route': 'western_m',
                                                                         'Western African Route': 'western_af'})

In [ ]:
df = df.rename(columns={'Total Number of Dead and Missing': 'Total_Deaths'})
df['date'] = pd.to_datetime(df['date'])
df.head()

,Route,date,flow,Total_Deaths,after,deaths_over_cross,ATT
0,central,2014-02-01,3335,9.0,0.0,0.002691,1.069824
1,central,2014-03-01,5550,1.0,0.0,0.000180,-4.842315
2,central,2014-04-01,15679,41.0,0.0,0.002608,-36.632737
3,central,2014-05-01,14597,299.0,0.0,0.020073,13.452974
4,central,2014-06-01,22778,314.0,0.0,0.013598,12.412410


In [ ]:
df

,Route,date,flow,Total_Deaths,after,deaths_over_cross,ATT
0,central,2014-02-01,3335,9.0,0.0,0.002691,1.069824
1,central,2014-03-01,5550,1.0,0.0,0.000180,-4.842315
2,central,2014-04-01,15679,41.0,0.0,0.002608,-36.632737
3,central,2014-05-01,14597,299.0,0.0,0.020073,13.452974
4,central,2014-06-01,22778,314.0,0.0,0.013598,12.412410
...,...,...,...,...,...,...,...
139,eastern,2016-10-01,4195,2.0,1.0,0.000477,-58.673449
140,eastern,2016-11-01,2680,14.0,1.0,0.005197,-65.243473
141,eastern,2016-12-01,2131,5.0,1.0,0.002341,-65.572272
142,eastern,2017-01-01,1826,1.0,1.0,0.000547,-69.903317


In [ ]:
conflicts = pd.read_csv("/content/drive/MyDrive/migration/conflicts_14_17_for_regression.csv")

In [ ]:
conflicts

,date,Route,fatalities
0,2014-02-01,central,57.0
1,2014-02-01,western_m,0.0
2,2014-02-01,eastern,0.0
3,2014-03-01,central,80.0
4,2014-03-01,western_m,0.0
...,...,...,...
106,2017-01-01,western_m,0.0
107,2017-01-01,eastern,113.0
108,2017-02-01,central,137.0
109,2017-02-01,western_m,0.0


In [ ]:
# ADD the african route !

conflicts['date'] = pd.to_datetime(conflicts['date'])
# Duplicate rows where 'Route' is 'western_af'
western_af_rows = conflicts[conflicts['Route'] == 'western_m'].copy()
# Modify the 'Route' value to 'western_m'
western_af_rows['Route'] = 'western_af'
# Append the modified rows to the original DataFrame
conflicts = conflicts.append(western_af_rows, ignore_index=True)
# Sort the DataFrame by 'date'
conflicts = conflicts.sort_values(by='date')

<ipython-input-22-003f36d7634d>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  conflicts = conflicts.append(western_af_rows, ignore_index=True)


In [ ]:
conflicts['date'] = pd.to_datetime(conflicts['date'], format='%Y-%m-%d')

In [ ]:
result_df = pd.merge(df, conflicts[['Route', 'date', 'fatalities']], left_on=['Route', 'date'], right_on=['Route', 'date'], how='left')

In [ ]:
result_df

,Route,date,flow,Total_Deaths,after,deaths_over_cross,ATT,Entropy,fatalities
0,central,2014-02-01,3335,9.0,0.0,0.002691,1.069824,3.368971,57.0
1,central,2014-03-01,5550,1.0,0.0,0.000180,-4.842315,3.622235,80.0
2,central,2014-04-01,15679,41.0,0.0,0.002608,-36.632737,2.683367,23.0
3,central,2014-05-01,14597,299.0,0.0,0.020073,13.452974,3.248205,159.0
4,central,2014-06-01,22778,314.0,0.0,0.013598,12.412410,3.256893,84.0
...,...,...,...,...,...,...,...,...,...
139,eastern,2016-10-01,4195,2.0,1.0,0.000477,-58.673449,2.946473,349.0
140,eastern,2016-11-01,2680,14.0,1.0,0.005197,-65.243473,3.555344,197.0
141,eastern,2016-12-01,2131,5.0,1.0,0.002341,-65.572272,4.005669,192.0
142,eastern,2017-01-01,1826,1.0,1.0,0.000547,-69.903317,3.816537,113.0


In [ ]:
# MODEL WITH THE WESTERN AFRICAN ROUTE

# the code chunk below applies one-way clustering by Route

model_ols_ent_wa = lm.PanelOLS.from_formula(
    formula="Total_Deaths ~ flow + ATT + Entropy + fatalities + EntityEffects + TimeEffects",
    data=result_df.set_index(["Route", "date"]),
  ).fit(cov_type="clustered", cluster_entity=True, cluster_time=False)

prettify_result(model_ols_ent_wa)

Panel OLS Model:
Total_Deaths ~ flow + ATT + Entropy + fatalities + EntityEffects + TimeEffects

Covariance Type: Clustered

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
flow           0.001       0.000        2.655    0.009
ATT            0.852       0.068       12.543    0.000
Entropy       18.615      26.763        0.696    0.488
fatalities     0.314       0.102        3.062    0.003

Included Fixed Effects:
        Total
Entity      4
Time       37

Summary statistics:
- Number of observations: 142
- R-squared (incl. FE): 0.583, Within R-squared: 0.142



/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
